In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

# EDA

In [ ]:
df.columns

In [3]:
## drop the columns that are not needed 
df_cleaned=df
df_cleaned=df_cleaned.drop(['url','image_url','county','VIN'],axis=1) 

In [ ]:
df_cleaned.isnull().sum()

In [5]:
# dropping rows due to NAs 

df_cleaned =df_cleaned.dropna(subset=['year','description','fuel','odometer','lat','long'])

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
len(df)-len(df_cleaned)

# Filling missing values 

In [ ]:
# filling missing values in title status with 'missing' 

df_cleaned['title_status'] = df_cleaned['title_status'].fillna('missing')
df_cleaned.isnull().sum()

In [11]:
df_cleaned['transmission'] = df_cleaned['transmission'].fillna('other')

In [ ]:
from model_extract import process_car_dataset

featured_eng=process_car_dataset(df_cleaned)



In [ ]:
increase = featured_eng['manufacturer'].value_counts() - df_cleaned['manufacturer'].value_counts()
print(increase)

In [ ]:
increase = featured_eng['drive'].value_counts() - df_cleaned['drive'].value_counts()
print(increase)

In [ ]:
increase = featured_eng['type'].value_counts() - df_cleaned['type'].value_counts()
print(increase)

In [ ]:
increase = featured_eng['cylinders'].value_counts() - df_cleaned['cylinders'].value_counts()
print(increase)